In [ ]:
# reference : https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [16]:
import glob

### read in all files in directory to a dataframe

In [20]:
pth = "/Users/zruxi/cyber_security_reading/examples/topic_model/data/medium/"


def read_file_to_text(filename):
    
    #file = "the-a-to-z-of-cyber-security-93150c4f336c.txt"

    lines = []
    with open(filename) as file:
        for l in file:
            lines.append(l.strip())
    text = ' '.join(lines[2:-2])
    
    return text


list_txt = []
for filename in glob.glob(pth + "*.txt"):
    
    text = read_file_to_text(filename)
    list_txt.append(text)
    
print(len(list_txt))

200


In [3]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/zruxi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Run in terminal or command prompt
#python3 -m spacy download en

### import packages

In [8]:

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Prep stop words

In [9]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Remove special characters

In [24]:
data = list_txt

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single/double quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\“", "", sent) for sent in data]
data = [re.sub("\‘", "", sent) for sent in data]

pprint(data[0])

('Computers are able to see, hear and learn. Welcome to the future” An '
 'existential problem for any Loan providers today is to find out the Loan '
 'applicants who are very likely to repay the loan. This way companies can '
 'avoid losses and incur huge profits. Home Credit offers easy, simple and '
 'fast loans for a range of Home Appliances, Mobile Phones, Laptops, Two '
 'Wheelers , and varied personal needs. Home Credit comes up with a Kaggle '
 'challenge to find out the loan applicants who is capable of repaying a loan, '
 'given the applicant data, all credits data from Credit Bureau, previous '
 'applications data from Home Credit and some more data. Building a model to '
 'predict how capable each applicant is of repaying a loan, so that '
 'sanctioning loan only for the applicants who are likely to repay the loan. '
 'Before starting any problem, it is better to lay the constraints. So that we '
 'can change the modelling process based on the constraints. 1. No strict '
 '

### Tokenize words and Clean-up text

In [25]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['computers', 'are', 'able', 'to', 'see', 'hear', 'and', 'learn', 'welcome', 'to', 'the', 'future', 'an', 'existential', 'problem', 'for', 'any', 'loan', 'providers', 'today', 'is', 'to', 'find', 'out', 'the', 'loan', 'applicants', 'who', 'are', 'very', 'likely', 'to', 'repay', 'the', 'loan', 'this', 'way', 'companies', 'can', 'avoid', 'losses', 'and', 'incur', 'huge', 'profits', 'home', 'credit', 'offers', 'easy', 'simple', 'and', 'fast', 'loans', 'for', 'range', 'of', 'home', 'appliances', 'mobile', 'phones', 'laptops', 'two', 'wheelers', 'and', 'varied', 'personal', 'needs', 'home', 'credit', 'comes', 'up', 'with', 'kaggle', 'challenge', 'to', 'find', 'out', 'the', 'loan', 'applicants', 'who', 'is', 'capable', 'of', 'repaying', 'loan', 'given', 'the', 'applicant', 'data', 'all', 'credits', 'data', 'from', 'credit', 'bureau', 'previous', 'applications', 'data', 'from', 'home', 'credit', 'and', 'some', 'more', 'data', 'building', 'model', 'to', 'predict', 'how', 'capable', 'each', 'a

### Creating Bigram and Trigram Models

In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['computers', 'are', 'able', 'to', 'see', 'hear', 'and', 'learn', 'welcome', 'to', 'the', 'future', 'an', 'existential', 'problem', 'for', 'any', 'loan', 'providers', 'today', 'is', 'to', 'find', 'out', 'the', 'loan', 'applicants', 'who', 'are', 'very', 'likely', 'to', 'repay', 'the', 'loan', 'this', 'way', 'companies', 'can', 'avoid', 'losses', 'and', 'incur', 'huge', 'profits', 'home', 'credit', 'offers', 'easy', 'simple', 'and', 'fast', 'loans', 'for', 'range', 'of', 'home', 'appliances', 'mobile_phones', 'laptops', 'two', 'wheelers', 'and', 'varied', 'personal', 'needs', 'home', 'credit', 'comes', 'up', 'with', 'kaggle', 'challenge', 'to', 'find', 'out', 'the', 'loan', 'applicants', 'who', 'is', 'capable', 'of', 'repaying', 'loan', 'given', 'the', 'applicant', 'data', 'all', 'credits', 'data', 'from', 'credit', 'bureau', 'previous', 'applications', 'data', 'from', 'home', 'credit', 'and', 'some', 'more', 'data', 'building', 'model', 'to', 'predict', 'how', 'capable', 'each', 'appli

### Remove Stopwords, Make Bigrams and Lemmatize

In [27]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [28]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['computer', 'able', 'see', 'hear', 'learn', 'welcome', 'future', 'existential', 'problem', 'loan', 'provider', 'today', 'find', 'loan', 'applicant', 'likely', 'repay', 'loan', 'way', 'company', 'avoid', 'loss', 'incur', 'huge', 'profit', 'home', 'credit', 'offer', 'easy', 'simple', 'fast', 'loan', 'range', 'home', 'appliance', 'laptop', 'wheeler', 'varied', 'personal', 'need', 'home', 'credit', 'come', 'find', 'loan', 'applicant', 'capable', 'repaying', 'loan', 'give', 'applicant', 'datum', 'credit', 'previous', 'application', 'home', 'credit', 'datum', 'building', 'model', 'predict', 'capable', 'applicant', 'repay', 'loan', 'sanction', 'loan', 'applicant', 'likely', 'repay', 'loan', 'start', 'problem', 'well', 'lie', 'constraint', 'change', 'modelling', 'process', 'base', 'constraint', 'strict', 'latency', 'constraint', 'give', 'loan', 'application', 'datum', 'predict', 'applicant', 'go', 'repay', 'loan', 'second', 'second', 'take', 'couple', 'minute', 'predict', 'even', 'hour', 'fi

### Create the Dictionary and Corpus needed for Topic Modeling

In [29]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 2), (3, 1), (4, 4), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 17), (11, 5), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 2), (19, 4), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 3), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 4), (37, 5), (38, 2), (39, 2), (40, 2), (41, 1), (42, 1), (43, 3), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 4), (51, 1), (52, 1), (53, 1), (54, 1), (55, 9), (56, 1), (57, 2), (58, 1), (59, 2), (60, 3), (61, 27), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 3), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 4), (90, 1), (91, 1), (92, 1), (93, 5), (94, 3), (95, 4), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 7), (108, 1), (109, 4), (110, 

In [30]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 3),
  ('academic', 1),
  ('accuracy', 2),
  ('achieve', 1),
  ('algorithm', 4),
  ('also', 2),
  ('amount', 1),
  ('analyticsvidhya', 1),
  ('appliance', 1),
  ('appliant', 1),
  ('applicant', 17),
  ('application', 5),
  ('apply', 1),
  ('arguably', 1),
  ('asp', 1),
  ('auc', 3),
  ('average', 1),
  ('avoid', 1),
  ('balance', 2),
  ('base', 4),
  ('binary', 1),
  ('blog', 1),
  ('boost', 3),
  ('building', 1),
  ('bureau', 1),
  ('business', 1),
  ('call', 1),
  ('cancer', 1),
  ('capability', 1),
  ('capable', 3),
  ('care', 1),
  ('case', 1),
  ('cash', 1),
  ('change', 1),
  ('chart', 1),
  ('class', 1),
  ('classification', 4),
  ('column', 5),
  ('com', 2),
  ('come', 2),
  ('comment', 2),
  ('commonly', 1),
  ('company', 1),
  ('compare', 3),
  ('complete', 1),
  ('computer', 1),
  ('concept', 1),
  ('connect', 1),
  ('consider', 1),
  ('consist', 1),
  ('constraint', 4),
  ('cost', 1),
  ('could', 1),
  ('couple', 1),
  ('create', 1),
  ('credit', 9),
  ('crown', 1

### Building the Topic Model

In [34]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### View the topics in LDA model

In [32]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"name" + 0.014*"log" + 0.013*"wordpiece" + 0.010*"reason" + '
  '0.008*"cybert" + 0.006*"scheme" + 0.006*"overlap" + 0.005*"ba" + '
  '0.005*"standardization" + 0.004*"attribution"'),
 (1,
  '0.017*"cti" + 0.013*"job" + 0.006*"graduate" + 0.006*"ask" + '
  '0.005*"skillset" + 0.005*"japanese" + 0.005*"husband" + 0.005*"resume" + '
  '0.005*"emphasize" + 0.005*"interview"'),
 (2,
  '0.016*"threat" + 0.011*"technique" + 0.010*"use" + 0.010*"detection" + '
  '0.010*"datum" + 0.009*"user" + 0.008*"security" + 0.008*"attack" + '
  '0.007*"information" + 0.007*"behavior"'),
 (3,
  '0.024*"model" + 0.023*"datum" + 0.012*"feature" + 0.011*"point" + '
  '0.011*"value" + 0.010*"use" + 0.009*"variable" + 0.009*"set" + 0.008*"see" '
  '+ 0.008*"number"'),
 (4,
  '0.020*"datum" + 0.013*"use" + 0.009*"detect" + 0.008*"different" + '
  '0.008*"match" + 0.008*"user" + 0.007*"example" + 0.007*"time" + '
  '0.007*"malicious" + 0.007*"algorithm"'),
 (5,
  '0.029*"model" + 0.021*"active" + 0

### Compute Model Perplexity and Coherence Score

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.475443872115852

Coherence Score:  0.345276468141126


### Visualize the topics-keywords

In [36]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
37    -0.217827 -0.180929       1        1  35.939213
33    -0.266150 -0.070040       2        1  19.051855
28    -0.328773  0.060796       3        1   8.577664
26    -0.287582 -0.057191       4        1   6.430146
38    -0.320944  0.047517       5        1   5.191122
2     -0.306272 -0.136721       6        1   4.705740
39    -0.275450 -0.101512       7        1   4.500434
35    -0.231077 -0.154767       8        1   2.712104
7     -0.206968 -0.105374       9        1   1.798782
3     -0.213172  0.159147      10        1   1.574240
9     -0.204660  0.229732      11        1   1.384361
44    -0.160119  0.014556      12        1   1.218444
49    -0.129596  0.166552      13        1   1.140704
21    -0.087134 -0.057401      14        1   0.975595
12    -0.130065  0.148387      15        1   0.862787
31    -0.088694 -0.043294      16        1   0.817367
43    -0.113628  0.155249      17        1   0.776121
14    -0.024161  0.157604      18        1   0.539795
45    -0.004981  0.033730      19        1   0.408490
5      0.018268  0.080076      20        1   0.325190
47     0.037263 -0.075997      21        1   0.283636
34     0.063070 -0.034712      22        1   0.241769
24     0.072235 -0.062800      23        1   0.197478
36     0.128359 -0.013088      24        1   0.148191
23     0.145471 -0.002771      25        1   0.054893
27     0.142724 -0.002823      26        1   0.046199
4      0.131618 -0.005858      27        1   0.016855
19     0.133585 -0.005064      28        1   0.016035
10     0.130971 -0.005816      29        1   0.013848
0      0.128074 -0.007033      30        1   0.009962
8      0.126498 -0.006211      31        1   0.005876
48     0.122853 -0.006584      32        1   0.001934
16     0.122980 -0.006527      33        1   0.001933
32     0.122921 -0.006652      34        1   0.001905
25     0.123066 -0.006571      35        1   0.001899
20     0.123049 -0.006490      36        1   0.001896
42     0.122972 -0.006517      37        1   0.001871
15     0.123112 -0.006529      38        1   0.001864
6      0.123025 -0.006543      39        1   0.001861
41     0.123101 -0.006543      40        1   0.001859
18     0.123153 -0.006518      41        1   0.001848
29     0.123158 -0.006433      42        1   0.001845
22     0.123105 -0.006595      43        1   0.001837
46     0.123219 -0.006517      44        1   0.001825
17     0.123189 -0.006488      45        1   0.001817
13     0.123199 -0.006441      46        1   0.001797
40     0.123239 -0.006497      47        1   0.001795
11     0.123217 -0.006507      48        1   0.001786
30     0.123326 -0.006472      49        1   0.001765
1      0.123233 -0.006523      50        1   0.001763, topic_info=         Term         Freq        Total Category  logprob  loglift
162     model  1419.000000  1419.000000  Default  30.0000  30.0000
870    threat   756.000000   756.000000  Default  29.0000  29.0000
829  security  1236.000000  1236.000000  Default  28.0000  28.0000
582    attack   709.000000   709.000000  Default  27.0000  27.0000
61      datum  1340.000000  1340.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
268       use     0.000281  1292.943970  Topic50  -9.0368  -4.3951
150      look     0.000281   317.036621  Topic50  -9.0369  -2.9895
42    company     0.000281   297.628143  Topic50  -9.0369  -2.9264
148       log     0.000281   196.650665  Topic50  -9.0382  -2.5132
243     start     0.000281   265.448029  Topic50  -9.0384  -2.8134

[3302 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5138     22  0.738467  abbreviation
0         1  0.457306          able
0         2  0.304871          able
0         4  0.092385          able
0         5  0.032335          able
...     ...       ...           ...
1247      5  0.0244

### Building LDA Mallet Model

In [50]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/zruxi/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=30, id2word=id2word)

In [51]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(14,
  [('strategy', 0.04365620736698499),
   ('match', 0.035811732605729875),
   ('information', 0.03274215552523874),
   ('source', 0.028308321964529332),
   ('datum', 0.02660300136425648),
   ('record', 0.02557980900409277),
   ('provide', 0.021487039563437928),
   ('data', 0.01637107776261937),
   ('apply', 0.015006821282401092),
   ('indictment', 0.01432469304229195)]),
 (29,
  [('security', 0.11922196796338673),
   ('cybersecurity', 0.033867276887871854),
   ('industry', 0.021739130434782608),
   ('people', 0.020137299771167048),
   ('job', 0.016933638443935927),
   ('skill', 0.016933638443935927),
   ('expert', 0.015102974828375287),
   ('work', 0.013958810068649886),
   ('team', 0.013729977116704805),
   ('resource', 0.012814645308924484)]),
 (5,
  [('datum', 0.06530711337703327),
   ('point', 0.03326050012138869),
   ('feature', 0.03180383588249575),
   ('variable', 0.026705511046370478),
   ('data', 0.02112163146394756),
   ('normal', 0.02015052197135227),
   ('anomaly_detec

In [58]:
data_lemmatized[0]

['computer',
 'able',
 'see',
 'hear',
 'learn',
 'welcome',
 'future',
 'existential',
 'problem',
 'loan',
 'provider',
 'today',
 'find',
 'loan',
 'applicant',
 'likely',
 'repay',
 'loan',
 'way',
 'company',
 'avoid',
 'loss',
 'incur',
 'huge',
 'profit',
 'home',
 'credit',
 'offer',
 'easy',
 'simple',
 'fast',
 'loan',
 'range',
 'home',
 'appliance',
 'laptop',
 'wheeler',
 'varied',
 'personal',
 'need',
 'home',
 'credit',
 'come',
 'find',
 'loan',
 'applicant',
 'capable',
 'repaying',
 'loan',
 'give',
 'applicant',
 'datum',
 'credit',
 'previous',
 'application',
 'home',
 'credit',
 'datum',
 'building',
 'model',
 'predict',
 'capable',
 'applicant',
 'repay',
 'loan',
 'sanction',
 'loan',
 'applicant',
 'likely',
 'repay',
 'loan',
 'start',
 'problem',
 'well',
 'lie',
 'constraint',
 'change',
 'modelling',
 'process',
 'base',
 'constraint',
 'strict',
 'latency',
 'constraint',
 'give',
 'loan',
 'application',
 'datum',
 'predict',
 'applicant',
 'go',
 'repa

In [52]:
mlmodel = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [53]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(mlmodel, corpus, id2word)
vis

/Users/zruxi/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/zruxi/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/zruxi/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8     -0.221334 -0.061997       1        1  6.292779
3      0.079943 -0.231166       2        1  4.752619
21    -0.048100  0.139082       3        1  4.193892
10     0.184060  0.109538       4        1  3.998232
25    -0.125951 -0.028795       5        1  3.961986
23    -0.235299  0.057823       6        1  3.954646
29    -0.148529 -0.061551       7        1  3.750528
9      0.136079  0.142811       8        1  3.718690
11     0.208698  0.025184       9        1  3.553065
0      0.108504  0.125579      10        1  3.547263
5      0.029587  0.140940      11        1  3.535098
12    -0.082047  0.085638      12        1  3.467257
26    -0.104641 -0.051767      13        1  3.273156
20     0.078087 -0.053315      14        1  3.216609
28    -0.110486  0.037125      15        1  3.195958
24     0.227467 -0.015201      16        1  3.162586
15     0.032705  0.004530      17        1  3.103205
22    -0.106268  0.213255      18        1  3.073447
19    -0.132010  0.030452      19        1  3.069375
7      0.184909 -0.053588      20        1  2.993025
27    -0.077245 -0.019932      21        1  2.958848
2      0.009203 -0.115493      22        1  2.896677
16     0.056357  0.005815      23        1  2.864505
14    -0.016031  0.143086      24        1  2.657331
17    -0.062268 -0.148758      25        1  2.641050
1     -0.015514 -0.172027      26        1  2.576517
4      0.000376  0.020503      27        1  2.541839
18    -0.024933 -0.104706      28        1  2.411137
6      0.132889 -0.075007      29        1  2.356748
13     0.041790 -0.088057      30        1  2.281929, topic_info=         Term        Freq        Total Category  logprob  loglift
162     model  1615.00000  1615.000000  Default  30.0000  30.0000
829  security  1362.00000  1362.000000  Default  29.0000  29.0000
870    threat   846.00000   846.000000  Default  28.0000  28.0000
582    attack   797.00000   797.000000  Default  27.0000  27.0000
539      user   680.00000   680.000000  Default  26.0000  26.0000
..        ...         ...          ...      ...      ...      ...
183    people    36.40320   277.993424  Topic30  -4.4316   1.7472
532      time    41.25696   761.360932  Topic30  -4.3064   0.8649
867     thing    31.54944   236.539902  Topic30  -4.5747   1.7656
19       base    26.69568   577.493909  Topic30  -4.7418   0.7060
508   similar    24.26880   155.650538  Topic30  -4.8371   1.9217

[2814 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
555       3  0.522578    ability
555      12  0.053324    ability
555      21  0.383935    ability
555      24  0.042659    ability
3645      6  0.971826   abnormal
...     ...       ...        ...
4719      2  0.970231      young
8480     22  0.832232  zeitgeist
3572     28  0.843339     zensor
5417     22  0.832232        zip
6301     16  0.871658        普通话

[4715 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 22, 11, 26, 24, 30, 10, 12, 1, 6, 13, 27, 21, 29, 25, 16, 23, 20, 8, 28, 3, 17, 15, 18, 2, 5, 19, 7, 14])